# Methodology
We will be creating our model with the help of Python so we start off by importing all the required packages

In [ ]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# import k-means for the clustering stage
from sklearn.cluster import KMeans

# DATA IMPORTING
Data is cleaned and arranged for processing.

In [3]:
df_besiktas = pd.read_csv('Istanbul.txt')
df_besiktas

,IL_ID,ILCE_ID,SEMT_ADI,SEMT_ID,POSTA_KODU
0,Istanbul,Bakırköy,Zeytinlik,1811,34140.0
1,Istanbul,Bakırköy,Cevizlik,1812,34142.0
2,Istanbul,Bakırköy,Kartaltepe,1813,34144.0
3,Istanbul,Bakırköy,Zuhuratbaba,1814,34147.0
4,Istanbul,Bakırköy,Yeşilköy,1815,34149.0
5,Istanbul,Bakırköy,Florya,1816,34153.0
6,Istanbul,Bakırköy,Ataköy,1817,34158.0
7,Istanbul,Bayrampaşa,Numunebağ,1818,34030.0
8,Istanbul,Bayrampaşa,Altıntepsi,1819,34035.0
9,Istanbul,Bayrampaşa,Muratpaşa,1820,34040.0


In [4]:
from geopy.geocoders import ArcGIS
geolocator = ArcGIS(scheme="https")

# Finding information Lat, Long for neighborhood

In [5]:
def get_x_y_uk(address1): 
    from geopy.geocoders import ArcGIS
    geolocator = ArcGIS(scheme="https")
    lat_coords = 0
    lng_coords = 0
    g=geolocator.geocode(address1)
    lng_coords = g[1][1]
    lat_coords = g[1][0]
    return str(lat_coords) +","+ str(lng_coords)

In [6]:
get_x_y_uk(34330)

'43.58668000000006,2.74869050500007'

In [13]:
geolocator.geocode('Beşiktaş,Akat Mahallesi')

Location(Akat, Beşiktaş, İstanbul, (41.08554000000004, 29.025630000000035, 0.0))

In [17]:
df_ilce_mah=df_besiktas['ILCE_ID']+', '+df_besiktas['SEMT_ADI'] + ' Mahallesi'
df_ilce_mah

0                   Bakırköy, Zeytinlik Mahallesi
1                    Bakırköy, Cevizlik Mahallesi
2                  Bakırköy, Kartaltepe Mahallesi
3                 Bakırköy, Zuhuratbaba Mahallesi
4                    Bakırköy, Yeşilköy Mahallesi
5                      Bakırköy, Florya Mahallesi
6                      Bakırköy, Ataköy Mahallesi
7                 Bayrampaşa, Numunebağ Mahallesi
8                Bayrampaşa, Altıntepsi Mahallesi
9                 Bayrampaşa, Muratpaşa Mahallesi
10                 Bayrampaşa, Yıldırım Mahallesi
11                     Beşiktaş, Levent Mahallesi
12                    Beşiktaş, Akatlar Mahallesi
13                     Beşiktaş, Etiler Mahallesi
14                    Beşiktaş, Levazım Mahallesi
15                      Beşiktaş, Bebek Mahallesi
16                 Beşiktaş, Arnavutköy Mahallesi
17                    Beşiktaş, Ortaköy Mahallesi
18                 Beşiktaş, Gayrettepe Mahallesi
19                   Beşiktaş, Abbasağa Mahallesi


In [18]:
coordinates_latlng_besiktas = df_ilce_mah.apply(lambda x: get_x_y_uk(x))
coordinates_latlng_besiktas

0        40.97538000000003,28.87235000000004
1       40.97656000000006,28.876940000000047
2        40.98571000000004,28.87599000000006
3       40.98738000000003,28.870390000000043
4       40.96013000000005,28.824780000000032
5       40.97299000000004,28.787930000000074
6        40.9919472330413,28.852413871499778
7       41.03268547492837,28.914072366397857
8       41.03926000000007,28.902950000000033
9       41.050470000000075,28.90686000000005
10      41.06638000000004,28.892660000000035
11      41.075860000000034,29.01720000000006
12     41.081300000000056,29.026040000000023
13      41.08247000000006,29.036510000000078
14       41.06399000000005,29.01890000000003
15      41.08111000000008,29.044160000000034
16     41.066140000000075,29.041540000000055
17       41.05150000000003,29.02771000000007
18      41.061090000000036,29.00711000000007
19      41.04759000000007,29.006210000000067
20      41.04973000000007,29.002180000000067
21      41.022500000000036,28.97144000000003
22      41

In [19]:
coordinates_latlng_besiktas

0        40.97538000000003,28.87235000000004
1       40.97656000000006,28.876940000000047
2        40.98571000000004,28.87599000000006
3       40.98738000000003,28.870390000000043
4       40.96013000000005,28.824780000000032
5       40.97299000000004,28.787930000000074
6        40.9919472330413,28.852413871499778
7       41.03268547492837,28.914072366397857
8       41.03926000000007,28.902950000000033
9       41.050470000000075,28.90686000000005
10      41.06638000000004,28.892660000000035
11      41.075860000000034,29.01720000000006
12     41.081300000000056,29.026040000000023
13      41.08247000000006,29.036510000000078
14       41.06399000000005,29.01890000000003
15      41.08111000000008,29.044160000000034
16     41.066140000000075,29.041540000000055
17       41.05150000000003,29.02771000000007
18      41.061090000000036,29.00711000000007
19      41.04759000000007,29.006210000000067
20      41.04973000000007,29.002180000000067
21      41.022500000000036,28.97144000000003
22      41

In [20]:
lat_besiktas = coordinates_latlng_besiktas.apply(lambda x: x.split(',')[0])
lat_besiktas

0       40.97538000000003
1       40.97656000000006
2       40.98571000000004
3       40.98738000000003
4       40.96013000000005
5       40.97299000000004
6        40.9919472330413
7       41.03268547492837
8       41.03926000000007
9      41.050470000000075
10      41.06638000000004
11     41.075860000000034
12     41.081300000000056
13      41.08247000000006
14      41.06399000000005
15      41.08111000000008
16     41.066140000000075
17      41.05150000000003
18     41.061090000000036
19      41.04759000000007
20      41.04973000000007
21     41.022500000000036
22      41.02725000000004
23      41.03219000000007
24      41.03923000000003
25      41.03283000000005
26     41.037000000000035
27      41.03667000000007
28      41.03385000000003
29      41.04929000000004
              ...        
263     40.98574000000008
264     41.01421000000005
265     40.98593000000005
266     41.02046000000007
267     41.03436000000005
268     41.01533000000006
269     41.03194000000008
270    41.04

In [22]:
lng_besiktas = coordinates_latlng_besiktas.apply(lambda x: x.split(',')[1])
lng_besiktas

0       28.87235000000004
1      28.876940000000047
2       28.87599000000006
3      28.870390000000043
4      28.824780000000032
5      28.787930000000074
6      28.852413871499778
7      28.914072366397857
8      28.902950000000033
9       28.90686000000005
10     28.892660000000035
11      29.01720000000006
12     29.026040000000023
13     29.036510000000078
14      29.01890000000003
15     29.044160000000034
16     29.041540000000055
17      29.02771000000007
18      29.00711000000007
19     29.006210000000067
20     29.002180000000067
21      28.97144000000003
22     28.979340000000036
23      28.98667000000006
24     28.967310000000055
25      28.98416000000003
26     28.980980000000045
27     28.985330000000033
28      28.96511000000004
29     28.949010000000044
              ...        
263    29.230300000000057
264     29.19440000000003
265    29.228650000000073
266    28.587290000000053
267     28.46025000000003
268    28.562430000000063
269    28.549360000000036
270    28.42

In [24]:
besiktas_merged = pd.concat([df_besiktas,lat_besiktas.astype(float), lng_besiktas.astype(float)], axis=1)
besiktas_merged.columns= ['city','town','borough','borough_ID','post_code','latitude','longitude']
besiktas_merged


,city,town,borough,borough_ID,post_code,latitude,longitude
0,Istanbul,Bakırköy,Zeytinlik,1811,34140.0,40.975380,28.872350
1,Istanbul,Bakırköy,Cevizlik,1812,34142.0,40.976560,28.876940
2,Istanbul,Bakırköy,Kartaltepe,1813,34144.0,40.985710,28.875990
3,Istanbul,Bakırköy,Zuhuratbaba,1814,34147.0,40.987380,28.870390
4,Istanbul,Bakırköy,Yeşilköy,1815,34149.0,40.960130,28.824780
5,Istanbul,Bakırköy,Florya,1816,34153.0,40.972990,28.787930
6,Istanbul,Bakırköy,Ataköy,1817,34158.0,40.991947,28.852414
7,Istanbul,Bayrampaşa,Numunebağ,1818,34030.0,41.032685,28.914072
8,Istanbul,Bayrampaşa,Altıntepsi,1819,34035.0,41.039260,28.902950
9,Istanbul,Bayrampaşa,Muratpaşa,1820,34040.0,41.050470,28.906860


In [25]:
besiktas_merged.dtypes


city           object
town           object
borough        object
borough_ID      int64
post_code     float64
latitude      float64
longitude     float64
dtype: object

# Visualize the Map of Istanbul


In [26]:
Besiktas = geolocator.geocode('Beşiktaş')
Besiktas_lng_coords = Besiktas[1][1]
Besiktas_lat_coords = Besiktas[1][0]


In [27]:
# Creating the map of London
map_Besiktas = folium.Map(location=[Besiktas_lat_coords, Besiktas_lng_coords], zoom_start=12)
# adding markers to map
for latitude, longitude, borough, town in zip(besiktas_merged['latitude'], besiktas_merged['longitude'], besiktas_merged['borough'], besiktas_merged['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Besiktas) 
map_Besiktas

In [28]:
CLIENT_ID = 'NC5E1OAAGOLRE2TRI4MLINGQJVJPFYDQXAOIZGYFJ1EVZNIO' 
CLIENT_SECRET = 'MBZ05TETAG31KX4AOCJIKGHMSWDFP0PYP4FQIYCZOXRFQI3Y'
VERSION = '20180605' # Foursquare API version

# Venues in Istanbul
To proceed with the next part, we need to define Foursquare API credentials.

Using Foursquare API, we are able to get the venue and venue categories around each neighbourhood in Istanbul.

In [29]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

Getting the venues in London

In [30]:
venues_in_Besiktas = getNearbyVenues(besiktas_merged['borough'], besiktas_merged['latitude'], besiktas_merged['longitude'])


Zeytinlik
Cevizlik
Kartaltepe
Zuhuratbaba
Yeşilköy
Florya
Ataköy
Numunebağ
Altıntepsi
Muratpaşa
Yıldırım
Levent
Akatlar
Etiler
Levazım
Bebek
Arnavutköy
Ortaköy
Gayrettepe
Abbasağa
Türkali
Arapcami
Kemankeş
Pürtelaş
İstiklal
Cihangir
Tarlabaşı
Taksim
Kasımpaşa
Halıcıoğlu
Arnavutköy
Boğazköy
Bolluca
Haraççı
Taşoluk
Hadımköy(Boyalık)
Durusu
Köyler
Eyüp
Rami
Alibeyköy
Yeşilpınar
Çırçır
Kemerburgaz
Göktürk
Köyler
Akdeniz
Küçükmustafapaşa
Fener
Karagümrük
Aksaray
Haseki
Kocamustafapaşa
Mevlanakapı
Yedikule
Eminönü/Cağaloğlu
Sirkeci
Tahtakale
Mahmutpaşa
Sultanahmet
Beyazıt
Kumkapı
Vefa
Yıldıztabya
Gaziosmanpaşa
Küçükköy
Fevziçakmak
Göztepe
Zincirlikuyu
Çağlayan
Kağıthane
Hamidiye
Gültepe
Çeliktepe
Şirintepe
Seyrantepe
Cennet
Sefaköy
Halkalı
Yarımburgaz
Köyler
Sarıyer
Büyükdere
Tarabya
İstinye
Yeniköy
Emirgan
Rumelihisarı
Bahçeköy
Köyler
39587
Meşrutiyet
Teşvikiye
Harbiye
Halaskargazi
Pangaltı
Bozkurt
Feriköy
Duatepe
Cumhuriyet
Şişli
Halideedip
Okmeydanı
Mecidiyeköy
Esentepe
Ayazağa
Maslak
Mal

In [31]:
venues_in_Besiktas.head()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Zeytinlik,40.97538,28.87235,The Coorner İstanbul,Café
1,Zeytinlik,40.97538,28.87235,Happy Moon's,Restaurant
2,Zeytinlik,40.97538,28.87235,M.A.C,Cosmetics Shop
3,Zeytinlik,40.97538,28.87235,Midpoint,Restaurant
4,Zeytinlik,40.97538,28.87235,MAX SPORT,Gym


In [32]:
venues_in_Besiktas.shape

(14187, 5)

# Grouping by Venue Categories
We need to now see how many Venue Categories are there for further processing

In [80]:
venues_in_Besiktas.groupby('Venue Category').max()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Advertising Agency,Gayrettepe,41.06311,29.00445,BiberSA
American Restaurant,Bebek,41.08111,29.04416,bebek waffle
Amphitheater,Levazım,41.06482,29.01955,Zorlu PSM Amfi
Antique Shop,Akatlar,36.99469,35.32401,Adana Sahaf Ve Plak Evi
Arcade,Akatlar,36.99469,35.32401,Şıkıdım Playstation
Art Gallery,Levent,41.08279,35.32401,İş Sanat Kibele Galerisi
Art Studio,Türkali,41.04892,29.00449,DC KOZMOS Sanat Akademileri Yapım Casting
Arts & Crafts Store,Etiler,41.08595,29.03743,Charm Studio
Arts & Entertainment,Arnavutköy,41.06719,29.04170,Doğulu Studios


# One Hot Encoding
We need to Encode our venue categories to get a better result for our clustering

In [33]:
Besiktas_venue_cat  = pd.get_dummies(venues_in_Besiktas[['Venue Category']], prefix="", prefix_sep="")
Besiktas_venue_cat 

,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,...,Waterfront,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Çöp Şiş Place
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Adding Neighbourhood into the mix.

In [34]:
Besiktas_venue_cat['Neighbourhood'] = venues_in_Besiktas['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Besiktas_venue_cat.columns[-1]] + list(Besiktas_venue_cat.columns[:-1])
Besiktas_venue_cat = Besiktas_venue_cat[fixed_columns]

Besiktas_venue_cat.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,...,Waterfront,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Çöp Şiş Place
0,Zeytinlik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Zeytinlik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Zeytinlik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Zeytinlik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Zeytinlik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Venue categories mean value
We will group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood

In [35]:
Besiktas_grouped = Besiktas_venue_cat.groupby('Neighbourhood').mean().reset_index()
Besiktas_grouped

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,...,Waterfront,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Çöp Şiş Place
0,39587,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.01,0.0,0.000000,0.010000,0.020000,0.00
1,Abbasağa,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.013158,0.00
2,Abdurrahmannafizgürman,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00
3,Acıbadem,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00
4,Akatlar,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00
5,Akdeniz,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00
6,Akfırat,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00
7,Aksaray,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.026316,0.000000,0.00
8,Akıncılar,0.016667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.016667,0.000000,0.000000,0.00
9,Alemdağ(Alemdar),0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00


to get the top most common venue categories

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


There are way too many venue categories, we can take the top 10 to cluster the neighbourhoods.

Creating a function to label the columns of the venue correctly

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Top venue categories
Getting the top venue categories in London

In [38]:
# create a new dataframe for Istanbul
neighborhoods_venues_sorted_besiktas = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_besiktas['Neighbourhood'] = Besiktas_grouped['Neighbourhood']

for ind in np.arange(Besiktas_grouped.shape[0]):
    neighborhoods_venues_sorted_besiktas.iloc[ind, 1:] = return_most_common_venues(Besiktas_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_besiktas.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,39587,Café,Coffee Shop,Restaurant,Clothing Store,Gym,Hotel,Boutique,Breakfast Spot,Bar,Yoga Studio
1,Abbasağa,Coffee Shop,Café,Performing Arts Venue,Burger Joint,Park,Bar,Hookah Bar,Lounge,Art Gallery,Historic Site
2,Abdurrahmannafizgürman,Café,Turkish Restaurant,Restaurant,Clothing Store,Gym,Department Store,Furniture / Home Store,Bakery,Soccer Field,Toy / Game Store
3,Acıbadem,Gym,Turkish Restaurant,Café,Coffee Shop,Convenience Store,Restaurant,Bakery,Park,Pharmacy,Manti Place
4,Akatlar,Café,Performing Arts Venue,Grocery Store,Kebab Restaurant,Park,Sporting Goods Shop,Stationery Store,Steakhouse,Fondue Restaurant,Music Venue


# Model Building
K Means
Let's cluster the city of london to roughly 5 to make it easier to analyze.

We use the K Means clustering technique to do so.

In [39]:
# set number of clusters
k_num_clusters = 5

Besiktas_grouped_clustering = Besiktas_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Besiktas = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Besiktas_grouped_clustering)
kmeans_Besiktas

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

# Labelling Clustered Data

In [90]:
kmeans_Besiktas.labels_

array([0, 1, 4, 0, 3, 1, 1, 2, 1, 0])

In [40]:
neighborhoods_venues_sorted_besiktas.insert(0, 'Cluster Labels', kmeans_Besiktas.labels_ +1)


In [41]:
Besiktas_data = besiktas_merged

Besiktas_data = Besiktas_data.join(neighborhoods_venues_sorted_besiktas.set_index('Neighbourhood'), on='borough')

Besiktas_data

,city,town,borough,borough_ID,post_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Istanbul,Bakırköy,Zeytinlik,1811,34140.0,40.975380,28.872350,3.0,Café,Coffee Shop,Restaurant,Turkish Restaurant,Clothing Store,Gym,Hotel,Seafood Restaurant,Cosmetics Shop,Gym / Fitness Center
1,Istanbul,Bakırköy,Cevizlik,1812,34142.0,40.976560,28.876940,1.0,Café,Coffee Shop,Seafood Restaurant,Turkish Restaurant,Bakery,Restaurant,Hotel,Gym,Kebab Restaurant,Fast Food Restaurant
2,Istanbul,Bakırköy,Kartaltepe,1813,34144.0,40.985710,28.875990,3.0,Bakery,Park,Turkish Restaurant,Gym,Steakhouse,Restaurant,Café,Fast Food Restaurant,Kebab Restaurant,Breakfast Spot
3,Istanbul,Bakırköy,Zuhuratbaba,1814,34147.0,40.987380,28.870390,3.0,Café,Bakery,Turkish Restaurant,Pizza Place,Gym,Burger Joint,Steakhouse,Gym / Fitness Center,Fast Food Restaurant,Mobile Phone Shop
4,Istanbul,Bakırköy,Yeşilköy,1815,34149.0,40.960130,28.824780,3.0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Turkish Restaurant,Ice Cream Shop,Dessert Shop,Manti Place,Steakhouse,Meyhane
5,Istanbul,Bakırköy,Florya,1816,34153.0,40.972990,28.787930,1.0,Café,Steakhouse,Hotel,Pool,Beach,Restaurant,Turkish Restaurant,Diner,Dessert Shop,Aquarium
6,Istanbul,Bakırköy,Ataköy,1817,34158.0,40.991947,28.852414,3.0,Basketball Stadium,Motorcycle Shop,Café,Food Truck,Coffee Shop,Seafood Restaurant,Toy / Game Store,Car Wash,Gym / Fitness Center,Park
7,Istanbul,Bayrampaşa,Numunebağ,1818,34030.0,41.032685,28.914072,3.0,Café,Clothing Store,Park,Bakery,Candy Store,Dessert Shop,Nightclub,Snack Place,Borek Place,Camera Store
8,Istanbul,Bayrampaşa,Altıntepsi,1819,34035.0,41.039260,28.902950,3.0,Turkish Restaurant,Dessert Shop,Arcade,Ice Cream Shop,Café,Tantuni Restaurant,Restaurant,Cosmetics Shop,Soccer Stadium,Furniture / Home Store
9,Istanbul,Bayrampaşa,Muratpaşa,1820,34040.0,41.050470,28.906860,3.0,Turkish Restaurant,Café,Arcade,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Bakery,Electronics Store,Park,Arts & Crafts Store


In [42]:
Besiktas_data_nonan = Besiktas_data.dropna(subset=['Cluster Labels'])
Besiktas_data_nonan

,city,town,borough,borough_ID,post_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Istanbul,Bakırköy,Zeytinlik,1811,34140.0,40.975380,28.872350,3.0,Café,Coffee Shop,Restaurant,Turkish Restaurant,Clothing Store,Gym,Hotel,Seafood Restaurant,Cosmetics Shop,Gym / Fitness Center
1,Istanbul,Bakırköy,Cevizlik,1812,34142.0,40.976560,28.876940,1.0,Café,Coffee Shop,Seafood Restaurant,Turkish Restaurant,Bakery,Restaurant,Hotel,Gym,Kebab Restaurant,Fast Food Restaurant
2,Istanbul,Bakırköy,Kartaltepe,1813,34144.0,40.985710,28.875990,3.0,Bakery,Park,Turkish Restaurant,Gym,Steakhouse,Restaurant,Café,Fast Food Restaurant,Kebab Restaurant,Breakfast Spot
3,Istanbul,Bakırköy,Zuhuratbaba,1814,34147.0,40.987380,28.870390,3.0,Café,Bakery,Turkish Restaurant,Pizza Place,Gym,Burger Joint,Steakhouse,Gym / Fitness Center,Fast Food Restaurant,Mobile Phone Shop
4,Istanbul,Bakırköy,Yeşilköy,1815,34149.0,40.960130,28.824780,3.0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Turkish Restaurant,Ice Cream Shop,Dessert Shop,Manti Place,Steakhouse,Meyhane
5,Istanbul,Bakırköy,Florya,1816,34153.0,40.972990,28.787930,1.0,Café,Steakhouse,Hotel,Pool,Beach,Restaurant,Turkish Restaurant,Diner,Dessert Shop,Aquarium
6,Istanbul,Bakırköy,Ataköy,1817,34158.0,40.991947,28.852414,3.0,Basketball Stadium,Motorcycle Shop,Café,Food Truck,Coffee Shop,Seafood Restaurant,Toy / Game Store,Car Wash,Gym / Fitness Center,Park
7,Istanbul,Bayrampaşa,Numunebağ,1818,34030.0,41.032685,28.914072,3.0,Café,Clothing Store,Park,Bakery,Candy Store,Dessert Shop,Nightclub,Snack Place,Borek Place,Camera Store
8,Istanbul,Bayrampaşa,Altıntepsi,1819,34035.0,41.039260,28.902950,3.0,Turkish Restaurant,Dessert Shop,Arcade,Ice Cream Shop,Café,Tantuni Restaurant,Restaurant,Cosmetics Shop,Soccer Stadium,Furniture / Home Store
9,Istanbul,Bayrampaşa,Muratpaşa,1820,34040.0,41.050470,28.906860,3.0,Turkish Restaurant,Café,Arcade,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Bakery,Electronics Store,Park,Arts & Crafts Store


# Visualizing the clustered neighbourhood
Let's plot the clusters

In [43]:
map_clusters_Besiktas = folium.Map(location=[Besiktas_lat_coords, Besiktas_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Besiktas_data_nonan['latitude'], Besiktas_data_nonan['longitude'], Besiktas_data_nonan['borough'], Besiktas_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_Besiktas)
        
map_clusters_Besiktas

In [54]:
Besiktas_data_nonan.loc[Besiktas_data_nonan['Cluster Labels'] == 1, Besiktas_data_nonan.columns[[1] + [2] +list(range(5, Besiktas_data_nonan.shape[1]))]]

,town,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bakırköy,Cevizlik,40.976560,28.876940,1.0,Café,Coffee Shop,Seafood Restaurant,Turkish Restaurant,Bakery,Restaurant,Hotel,Gym,Kebab Restaurant,Fast Food Restaurant
5,Bakırköy,Florya,40.972990,28.787930,1.0,Café,Steakhouse,Hotel,Pool,Beach,Restaurant,Turkish Restaurant,Diner,Dessert Shop,Aquarium
10,Bayrampaşa,Yıldırım,41.066380,28.892660,1.0,Café,Bakery,Turkish Restaurant,Electronics Store,Pie Shop,Dessert Shop,Coffee Shop,Fast Food Restaurant,Tea Room,Gym
19,Beşiktaş,Abbasağa,41.047590,29.006210,1.0,Coffee Shop,Café,Performing Arts Venue,Burger Joint,Park,Bar,Hookah Bar,Lounge,Art Gallery,Historic Site
20,Beşiktaş,Türkali,41.049730,29.002180,1.0,Coffee Shop,Café,Park,Soccer Stadium,Historic Site,Performing Arts Venue,Lounge,Gym,Bistro,Pub
28,Beyoğlu,Kasımpaşa,41.033850,28.965110,1.0,Café,Electronics Store,Turkish Restaurant,Arcade,Bakery,Restaurant,Boat or Ferry,Soup Place,Snack Place,Plaza
29,Beyoğlu,Halıcıoğlu,41.049290,28.949010,1.0,Café,Pie Shop,Bakery,Department Store,Soccer Stadium,Breakfast Spot,Gym,Music Venue,BBQ Joint,Hookah Bar
35,Arnavutköy,Hadımköy(Boyalık),41.155560,28.625280,1.0,Café,Soccer Field,Park,Steakhouse,Convenience Store,Rental Car Location,Turkish Restaurant,Pedestrian Plaza,Bus Stop,Coffee Shop
37,Arnavutköy,Köyler,41.185580,28.741470,1.0,Café,Turkish Restaurant,Dessert Shop,Restaurant,Seafood Restaurant,Gym,Gym / Fitness Center,Bakery,Fast Food Restaurant,Steakhouse
40,Eyüp,Alibeyköy,41.076840,28.944180,1.0,Café,Dessert Shop,Turkish Restaurant,Kebab Restaurant,Soccer Stadium,Coffee Shop,Restaurant,Steakhouse,Tea Room,Spa


In [56]:

Besiktas_data_nonan.loc[Besiktas_data_nonan['Cluster Labels'] == 2, Besiktas_data_nonan.columns[[1] + [2]+list(range(5, Besiktas_data_nonan.shape[1]))]]

,town,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
117,Güngören,Sanayi,37.80014,30.56933,2.0,Farm,Furniture / Home Store,Business Service,Frozen Yogurt Shop,Event Space,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Field
141,Esenler,Sanayi,37.80014,30.56933,2.0,Farm,Furniture / Home Store,Business Service,Frozen Yogurt Shop,Event Space,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Field
159,Sultangazi,Habipler,41.13363,28.83951,2.0,Convenience Store,Farm,Turkish Restaurant,Pizza Place,Construction & Landscaping,Athletics & Sports,Brewery,Moving Target,Soup Place,Seafood Restaurant
217,Ümraniye,Reşadiye,40.27994,30.29969,2.0,Farm,Mountain,Water Park,Çöp Şiş Place,Flea Market,Event Space,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant
273,Çatalca,Binkılıç,41.41062,28.19006,2.0,Convenience Store,Bakery,Forest,Soccer Stadium,Beach,Mountain,Plaza,Garden,Farm,Cruise Ship
274,Çatalca,Çiftlikköy,41.34036,28.40755,2.0,Convenience Store,Night Market,Soccer Stadium,Arts & Entertainment,Diner,Café,Village,Mountain,Tree,Forest


In [57]:

Besiktas_data_nonan.loc[Besiktas_data_nonan['Cluster Labels'] == 3, Besiktas_data_nonan.columns[[1] + [2]+list(range(5, Besiktas_data_nonan.shape[1]))]]

,town,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bakırköy,Zeytinlik,40.975380,28.872350,3.0,Café,Coffee Shop,Restaurant,Turkish Restaurant,Clothing Store,Gym,Hotel,Seafood Restaurant,Cosmetics Shop,Gym / Fitness Center
2,Bakırköy,Kartaltepe,40.985710,28.875990,3.0,Bakery,Park,Turkish Restaurant,Gym,Steakhouse,Restaurant,Café,Fast Food Restaurant,Kebab Restaurant,Breakfast Spot
3,Bakırköy,Zuhuratbaba,40.987380,28.870390,3.0,Café,Bakery,Turkish Restaurant,Pizza Place,Gym,Burger Joint,Steakhouse,Gym / Fitness Center,Fast Food Restaurant,Mobile Phone Shop
4,Bakırköy,Yeşilköy,40.960130,28.824780,3.0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Turkish Restaurant,Ice Cream Shop,Dessert Shop,Manti Place,Steakhouse,Meyhane
6,Bakırköy,Ataköy,40.991947,28.852414,3.0,Basketball Stadium,Motorcycle Shop,Café,Food Truck,Coffee Shop,Seafood Restaurant,Toy / Game Store,Car Wash,Gym / Fitness Center,Park
7,Bayrampaşa,Numunebağ,41.032685,28.914072,3.0,Café,Clothing Store,Park,Bakery,Candy Store,Dessert Shop,Nightclub,Snack Place,Borek Place,Camera Store
8,Bayrampaşa,Altıntepsi,41.039260,28.902950,3.0,Turkish Restaurant,Dessert Shop,Arcade,Ice Cream Shop,Café,Tantuni Restaurant,Restaurant,Cosmetics Shop,Soccer Stadium,Furniture / Home Store
9,Bayrampaşa,Muratpaşa,41.050470,28.906860,3.0,Turkish Restaurant,Café,Arcade,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Bakery,Electronics Store,Park,Arts & Crafts Store
11,Beşiktaş,Levent,41.075860,29.017200,3.0,Nightclub,Coffee Shop,Sporting Goods Shop,Hotel,Café,Gym / Fitness Center,Turkish Restaurant,Spa,Health & Beauty Service,Steakhouse
12,Beşiktaş,Akatlar,41.081300,29.026040,3.0,Café,Performing Arts Venue,Grocery Store,Kebab Restaurant,Park,Sporting Goods Shop,Stationery Store,Steakhouse,Fondue Restaurant,Music Venue


In [47]:
Besiktas_data_nonan.loc[Besiktas_data_nonan['Cluster Labels'] == 4, Besiktas_data_nonan.columns[[1] + list(range(5, Besiktas_data_nonan.shape[1]))]]

,town,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Eyüp,40.27155,37.47101,4.0,Mountain,Çöp Şiş Place,Flea Market,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field
111,Avcılar,37.35492,28.63444,4.0,Mountain,Waterfall,Çöp Şiş Place,Flea Market,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant
235,Tuzla,38.96213,31.10054,4.0,Bus Station,Mountain,Çöp Şiş Place,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


In [48]:
Besiktas_data_nonan.loc[Besiktas_data_nonan['Cluster Labels'] == 5, Besiktas_data_nonan.columns[[1] + list(range(5, Besiktas_data_nonan.shape[1]))]]

,town,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Fatih,36.82352,34.81513,5.0,Shopping Mall,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field


# Results and Discussion
The neighbourhoods of Istanbul are very mulitcultural. There are a lot of different cusines including Syrian, Italian, Turkish and Chinese. Istanbul seems to take a step further in this direction by having a lot of Restaurants, bars, juice bars, coffee shops, Fish and Chips shop and Breakfast spots. It has a lot of shopping options too with that of the Flea markets, flower shops, fish markets, Fishing stores, clothing stores. The main modes of transport seem to be Buses and trains. For leisure, the neighbourhoods are set up to have lots of parks, gyms and Historic sites.

Overall, the city of Istanbul offers a multicultural, diverse and certainly an entertaining experience.




# Conclusion
The purpose of this project was to explore the cities of Istanbul and see how attractive it is to potential tourists and migrants. We explored Istanbul based on their postal codes and then extrapolated the common venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together.

We could see that each of the neighbourhoods in both the cities have a wide variety of experiences to offer which is unique in it's own way. The cultural diversity is quite evident which also gives the feeling of a sense of inclusion.

